In [22]:
import sys
sys.path.append('../')
from localconfig import BASE_PATH,EXPS

EXP=EXPS[0]
print(EXP)
print(BASE_PATH)

import pathlib
import numpy as np
np.set_printoptions(threshold=1050)
from pprint import pprint

import maxone_code.data

takahashi1112
/Users/KozSakk/Documents/research/sakurayama/201911artifact_removal


In [23]:
import importlib
importlib.reload(maxone_code.data)
from maxone_code.data import RawFile,elecid2posi
from maxone_code.data import Experiment

In [24]:
path=pathlib.Path(BASE_PATH).joinpath('data',EXP,'raw','searching-5764-5764.raw.h5')
exp=Experiment(path)
exp.disp()

sig shape: (1028, 120000)
record time: 6[s]
record eles:size 25, [6203, 5765, 5546, 5323, 5324, 5543, 5766, 5985, 5542, 5983, 5762, 5545, 5984, 5764, 6204, 5982, 6205, 6206, 5326, 5763, 5322, 6202, 5325, 5986, 5544]
record channels: dict_keys([60, 72, 84, 88, 180, 192, 196, 208, 436, 536, 544, 548, 564, 600, 612, 636, 700, 712, 716, 720, 732, 740, 744, 752, 756])
trail start frame  [20290, 40370, 60450, 80530, 100610]
stim interval  [20080 20080 20080 20080]


In [51]:
STIM_ELEC=path.name.split('.')[0].split('-')[-1]
STIM_CHANNEL=exp.rawfile.elec2channel(int(STIM_ELEC))

SIZE_TRIAL=5
SIZE_CHANNEL=24
SIZE_FRAME=80 #for 4ms

forcus_channels=list(exp.rawfile.channel2elec_table.keys())
forcus_channels.remove(STIM_CHANNEL)

assert(SIZE_TRIAL==len(exp.trial_start_frames))
assert(SIZE_CHANNEL==len(forcus_channels))

channel2data_table={}
data2channel_table={}
for i,chan in enumerate(forcus_channels):
    channel2data_table[chan]=i
    data2channel_table[i]=chan

data_raw=np.zeros((SIZE_TRIAL,SIZE_CHANNEL,SIZE_FRAME))
source=exp.rawfile.sig

for i,start_frame in enumerate(exp.trial_start_frames):
    for j,channel in enumerate(forcus_channels):
        s_slice=np.s_[channel, start_frame:start_frame+SIZE_FRAME]
        d_slice=np.s_[:SIZE_FRAME]
        source.read_direct(data_raw[i][j], source_sel=s_slice, dest_sel=d_slice)
